<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Michael", "transactions": [{"transaction-id": 759, "amount": 286}, {"transaction-id": 1791, "amount": 257}, {"transaction-id": 2296, "amount": 252}, {"transaction-id": 3054, "amount": 236}, {"transaction-id": 4301, "amount": 263}, {"transaction-id": 4614, "amount": 271}, {"transaction-id": 4752, "amount": 270}, {"transaction-id": 5251, "amount": 255}, {"transaction-id": 6341, "amount": 251}, {"transaction-id": 6497, "amount": 252}, {"transaction-id": 6824, "amount": 247}, {"transaction-id": 6864, "amount": 257}, {"transaction-id": 7717, "amount": 248}, {"transaction-id": 8013, "amount": 271}, {"transaction-id": 9220, "amount": 265}]}\n',)

In [5]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [6]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [7]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [8]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Michael", "transactions": [{"transaction-id": 759, "amount": 286}, {"transaction-id": 1791, "amount": 257}, {"transaction-id": 2296, "amount": 252}, {"transaction-id": 3054, "amount": 236}, {"transaction-id": 4301, "amount": 263}, {"transaction-id": 4614, "amount": 271}, {"transaction-id": 4752, "amount": 270}, {"transaction-id": 5251, "amount": 255}, {"transaction-id": 6341, "amount": 251}, {"transaction-id": 6497, "amount": 252}, {"transaction-id": 6824, "amount": 247}, {"transaction-id": 6864, "amount": 257}, {"transaction-id": 7717, "amount": 248}, {"transaction-id": 8013, "amount": 271}, {"transaction-id": 9220, "amount": 265}]}\n',
 '{"id": 1, "name": "Jerry", "transactions": [{"transaction-id": 166, "amount": 22}, {"transaction-id": 270, "amount": -88}, {"transaction-id": 1118, "amount": 46}, {"transaction-id": 2694, "amount": 146}, {"transaction-id": 3026, "amount": 37}, {"transaction-id": 3145, "amount": 24}, {"transaction-id": 3219, "amount": 95}, {"trans

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [9]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Michael',
  'transactions': [{'transaction-id': 759, 'amount': 286},
   {'transaction-id': 1791, 'amount': 257},
   {'transaction-id': 2296, 'amount': 252},
   {'transaction-id': 3054, 'amount': 236},
   {'transaction-id': 4301, 'amount': 263},
   {'transaction-id': 4614, 'amount': 271},
   {'transaction-id': 4752, 'amount': 270},
   {'transaction-id': 5251, 'amount': 255},
   {'transaction-id': 6341, 'amount': 251},
   {'transaction-id': 6497, 'amount': 252},
   {'transaction-id': 6824, 'amount': 247},
   {'transaction-id': 6864, 'amount': 257},
   {'transaction-id': 7717, 'amount': 248},
   {'transaction-id': 8013, 'amount': 271},
   {'transaction-id': 9220, 'amount': 265}]},
 {'id': 1,
  'name': 'Jerry',
  'transactions': [{'transaction-id': 166, 'amount': 22},
   {'transaction-id': 270, 'amount': -88},
   {'transaction-id': 1118, 'amount': 46},
   {'transaction-id': 2694, 'amount': 146},
   {'transaction-id': 3026, 'amount': 37},
   {'transaction-id': 3145, 'a

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [10]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 20,
  'name': 'Alice',
  'transactions': [{'transaction-id': 768, 'amount': 1828},
   {'transaction-id': 1013, 'amount': 1919},
   {'transaction-id': 1088, 'amount': 1673},
   {'transaction-id': 1232, 'amount': 1751},
   {'transaction-id': 1669, 'amount': 1775},
   {'transaction-id': 1900, 'amount': 1945},
   {'transaction-id': 2241, 'amount': 1750},
   {'transaction-id': 2543, 'amount': 1817},
   {'transaction-id': 2819, 'amount': 1727},
   {'transaction-id': 3013, 'amount': 1816},
   {'transaction-id': 3428, 'amount': 1891},
   {'transaction-id': 3632, 'amount': 1857},
   {'transaction-id': 3802, 'amount': 1814},
   {'transaction-id': 4617, 'amount': 1818},
   {'transaction-id': 4965, 'amount': 1781},
   {'transaction-id': 6895, 'amount': 1753},
   {'transaction-id': 6964, 'amount': 1809},
   {'transaction-id': 7699, 'amount': 1801},
   {'transaction-id': 7764, 'amount': 1747},
   {'transaction-id': 7985, 'amount': 1933},
   {'transaction-id': 8509, 'amount': 1915},
   {'tran

In [11]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 26},
 {'name': 'Alice', 'count': 5},
 {'name': 'Alice', 'count': 71},
 {'name': 'Alice', 'count': 13},
 {'name': 'Alice', 'count': 71})

In [12]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(26, 5, 71, 13, 71)

In [13]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

68.35368956743002

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [14]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 768, 'amount': 1828},
  {'transaction-id': 1013, 'amount': 1919},
  {'transaction-id': 1088, 'amount': 1673},
  {'transaction-id': 1232, 'amount': 1751},
  {'transaction-id': 1669, 'amount': 1775},
  {'transaction-id': 1900, 'amount': 1945},
  {'transaction-id': 2241, 'amount': 1750},
  {'transaction-id': 2543, 'amount': 1817},
  {'transaction-id': 2819, 'amount': 1727},
  {'transaction-id': 3013, 'amount': 1816},
  {'transaction-id': 3428, 'amount': 1891},
  {'transaction-id': 3632, 'amount': 1857},
  {'transaction-id': 3802, 'amount': 1814},
  {'transaction-id': 4617, 'amount': 1818},
  {'transaction-id': 4965, 'amount': 1781},
  {'transaction-id': 6895, 'amount': 1753},
  {'transaction-id': 6964, 'amount': 1809},
  {'transaction-id': 7699, 'amount': 1801},
  {'transaction-id': 7764, 'amount': 1747},
  {'transaction-id': 7985, 'amount': 1933},
  {'transaction-id': 8509, 'amount': 1915},
  {'transaction-id': 8732, 'amount': 1855},
  {'transaction-id': 8840, 'amoun

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 768, 'amount': 1828},
 {'transaction-id': 1013, 'amount': 1919},
 {'transaction-id': 1088, 'amount': 1673})

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(1828, 1919, 1673)

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1119.7633920262072

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [18]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [19]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [20]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [21]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [22]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 86), ('Alice', 93), ('Alice', 103), ('Alice', 111), ('Bob', 110), ('Bob', 120), ('Bob', 129), ('Bob', 140), ('Charlie', 64), ('Charlie', 72), ('Charlie', 77), ('Charlie', 83), ('Dan', 107), ('Dan', 138), ('Dan', 244), ('Edith', 77), ('Edith', 84), ('Edith', 91), ('Edith', 98), ('Frank', 130), ('Frank', 142), ('Frank', 152), ('Frank', 167), ('George', 110), ('George', 120), ('George', 269), ('Hannah', 152), ('Hannah', 164), ('Hannah', 179), ('Hannah', 191), ('Ingrid', 139), ('Ingrid', 155), ('Ingrid', 163), ('Ingrid', 176), ('Jerry', 87), ('Jerry', 95), ('Jerry', 101), ('Jerry', 109), ('Kevin', 110), ('Kevin', 120), ('Kevin', 130), ('Kevin', 140), ('Laura', 102), ('Laura', 114), ('Laura', 124), ('Laura', 133), ('Michael', 168), ('Michael', 185), ('Michael', 197), ('Michael', 217), ('Norbert', 121), ('Norbert', 132), ('Norbert', 143), ('Norbert', 153), ('Oliver', 108), ('Oliver', 120), ('Oliver', 130), ('Oliver', 140), ('Patricia', 88), ('Patricia', 96), ('Patricia', 104), ('P

In [23]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 393), ('Bob', 499), ('Charlie', 296), ('Dan', 489), ('Edith', 350), ('Frank', 591), ('George', 499), ('Hannah', 686), ('Ingrid', 633), ('Jerry', 392), ('Kevin', 500), ('Laura', 473), ('Michael', 767), ('Norbert', 549), ('Oliver', 498), ('Patricia', 400), ('Quinn', 696), ('Ray', 150), ('Sarah', 200), ('Tim', 600), ('Ursula', 693), ('Victor', 249), ('Wendy', 350), ('Xavier', 400), ('Yvonne', 600), ('Zelda', 400)]
CPU times: user 162 ms, sys: 7.84 ms, total: 170 ms
Wall time: 625 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [24]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [25]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Michael,"[{'transaction-id': 759, 'amount': 286}, {'tra..."
1,1,Jerry,"[{'transaction-id': 166, 'amount': 22}, {'tran..."
2,2,Dan,"[{'transaction-id': 573, 'amount': -1086}, {'t..."
3,3,Kevin,"[{'transaction-id': 203, 'amount': 22}, {'tran..."
4,4,Hannah,"[{'transaction-id': 65, 'amount': 474}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [26]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 265 ms, sys: 8.26 ms, total: 273 ms
Wall time: 1.87 s


name
Alice      393
Bob        499
Charlie    296
Dan        489
Edith      350
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [27]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Michael', 'amount': 286, 'transaction-id': 759},
 {'id': 0, 'name': 'Michael', 'amount': 257, 'transaction-id': 1791},
 {'id': 0, 'name': 'Michael', 'amount': 252, 'transaction-id': 2296})

In [28]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Michael,286,759
1,0,Michael,257,1791
2,0,Michael,252,2296
3,0,Michael,236,3054
4,0,Michael,263,4301


In [29]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 254 ms, sys: 13.4 ms, total: 267 ms
Wall time: 1.59 s


name
Alice       26863
Bob         17223
Charlie     13838
Dan         20982
Edith        9193
Frank       34827
George      28393
Hannah      29369
Ingrid      18945
Jerry       11229
Kevin       20834
Laura       19839
Michael     25624
Norbert     24978
Oliver      17791
Patricia    16150
Quinn       31467
Ray          2662
Sarah        7711
Tim         18038
Ursula      25161
Victor       5511
Wendy       20899
Xavier      14309
Yvonne      24505
Zelda       13659
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [30]:
client.shutdown()